In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
import preprocess.preprocess as cf

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


In [ ]:
data_path = os.path.abspath(os.path.join(os.getcwd(), 'data'))

df_traveller = pd.read_csv(os.path.join(data_path, 'traveller.csv'))
df_travel = pd.read_csv(os.path.join(data_path, 'travel.csv'))
df_area = pd.read_csv(os.path.join(data_path, 'area.csv'))

In [ ]:
# features = [
#     'TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
#     'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
#     'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM',
#     'VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD', 'SIDO', 'GUNGU', 'RESIDENCE_TIME_MIN', 'REVISIT_YN',
#     'DGSTFN'
# ]

df = cf.cf_preprocess(df_traveller, df_travel, df_area)

df

In [23]:
a = df[df['TRAVELER_ID']=='a004293']
a[['X_COORD', 'Y_COORD']]

KeyError: 'X_COORD'

In [20]:
a = df['SIDO'].unique()
a

array(['경기', '인천', '서울', '강원', '충북', '서울특별시', '경북', '충남', '인천광역시', '경기도',
       '대전', '경상북도', '대구', '광주', '경남', '세종특별자치시', '부산', '울산', '강원도', '전남',
       '전북', '부산광역시', '대구광역시', '울산광역시', '경상남도', '광복동', '제주특별자치도', '충청남도',
       '전라북도', '충청북도', '전라남도', '광주광역시', '동부리', '대전광역시'], dtype=object)

In [11]:
class TravelDataset(Dataset):
    def __init__(self, df):
        self.users = df['TRAVELER_ID'].astype('category').cat.codes.values
        self.items = df['VISIT_AREA_NM'].astype('category').cat.codes.values
        self.visit_area_type_cd = df['VISIT_AREA_TYPE_CD'].astype('category').cat.codes.values
        self.sido = df['SIDO'].astype('category').cat.codes.values
        self.gungu = df['GUNGU'].astype('category').cat.codes.values
        self.residence_time_min = df['RESIDENCE_TIME_MIN'].values
        self.revisit_yn = df['REVISIT_YN'].values
        self.travel_mission_priority = df['TRAVEL_MISSION_PRIORITY'].astype('category').cat.codes.values
        self.gender = df['GENDER'].astype('category').values
        self.age_grp = df['AGE_GRP'].astype('category').cat.codes.values
        self.income = df['INCOME'].astype('category').cat.codes.values
        self.style1 = df['TRAVEL_STYL_1'].astype('category').cat.codes.values
        self.style2 = df['TRAVEL_STYL_2'].astype('category').cat.codes.values
        self.style3 = df['TRAVEL_STYL_3'].astype('category').cat.codes.values
        self.style4 = df['TRAVEL_STYL_4'].astype('category').cat.codes.values
        self.style5 = df['TRAVEL_STYL_5'].astype('category').cat.codes.values
        self.style6 = df['TRAVEL_STYL_6'].astype('category').cat.codes.values
        self.style7 = df['TRAVEL_STYL_7'].astype('category').cat.codes.values
        self.style8 = df['TRAVEL_STYL_8'].astype('category').cat.codes.values
        self.motive = df['TRAVEL_MOTIVE_1'].astype('category').cat.codes.values
        self.travel_num = df['TRAVEL_NUM'].values
        self.companion_num = df['TRAVEL_COMPANIONS_NUM'].values
        
        self.ratings = df['SCORE'].values

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        # 모든 특성을 포함시켜 반환
        features = (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.visit_area_type_cd[idx], dtype=torch.long),
            torch.tensor(self.sido[idx], dtype=torch.long),
            torch.tensor(self.gungu[idx], dtype=torch.long),
            torch.tensor(self.residence_time_min[idx], dtype=torch.long),
            torch.tensor(self.revisit_yn[idx], dtype=torch.long),
            torch.tensor(self.travel_mission_priority[idx], dtype=torch.long),
            torch.tensor(self.gender[idx], dtype=torch.long),
            torch.tensor(self.age_grp[idx], dtype=torch.long),
            torch.tensor(self.income[idx], dtype=torch.long),
            torch.tensor(self.style1[idx], dtype=torch.long),
            torch.tensor(self.style2[idx], dtype=torch.long),
            torch.tensor(self.style3[idx], dtype=torch.long),
            torch.tensor(self.style4[idx], dtype=torch.long),
            torch.tensor(self.style5[idx], dtype=torch.long),
            torch.tensor(self.style6[idx], dtype=torch.long),
            torch.tensor(self.style7[idx], dtype=torch.long),
            torch.tensor(self.style8[idx], dtype=torch.long),
            torch.tensor(self.motive[idx], dtype=torch.long),
            torch.tensor(self.travel_num[idx], dtype=torch.long),
            torch.tensor(self.companion_num[idx], dtype=torch.long)
        )

        target = torch.tensor(self.ratings[idx], dtype=torch.float)

        return features, target

In [12]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items, num_sido, num_gungu, num_travel_mission_priority, num_age_grp, num_income,
                 num_style1, num_style2, num_style3, num_style4, num_style5, num_style6, num_style7, num_style8, num_motive, embed_size):
        super(NCF, self).__init__()
        # 기본 임베딩
        self.user_embed = nn.Embedding(num_users, embed_size)
        self.item_embed = nn.Embedding(num_items, embed_size)
        
        # 추가적인 범주형 특성에 대한 임베딩
        self.sido_embed = nn.Embedding(num_sido, embed_size)
        self.gungu_embed = nn.Embedding(num_gungu, embed_size)
        self.travel_mission_priority_embed = nn.Embedding(num_travel_mission_priority, embed_size)
        self.age_grp_embed = nn.Embedding(num_age_grp, embed_size)
        self.income_embed = nn.Embedding(num_income, embed_size)
        #self.style1_embed = nn.Embedding(num_style1, embed_size)
        self.style2_embed = nn.Embedding(num_style2, embed_size)
        self.style3_embed = nn.Embedding(num_style3, embed_size)
        self.style4_embed = nn.Embedding(num_style4, embed_size)
        self.style5_embed = nn.Embedding(num_style5, embed_size)
        self.style6_embed = nn.Embedding(num_style6, embed_size)
        self.style7_embed = nn.Embedding(num_style7, embed_size)
        self.style8_embed = nn.Embedding(num_style8, embed_size)
        self.motive_embed = nn.Embedding(num_motive, embed_size)
        
        # 연속형 특성을 위한 입력 크기
        continuous_feature_input_dim = 5  # 'gender', 'revisit_yn', 'residence_time_min', 'travel_num', 'companion_num'
        
        # MLP 파트
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_size * 15 + continuous_feature_input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, features):
        user_ids, item_ids, sido, gungu, residence_time_min, revisit_yn, travel_mission_priority, gender, age_grp, income, style1, style2, style3, style4, style5, style6, style7, style8, motive, residence_time_min, travel_num, companion_num = features
        
        # 기본 및 추가적인 범주형 특성에 대한 임베딩
        user_embedded = self.user_embed(user_ids)
        item_embedded = self.item_embed(item_ids)
        sido_embedded = self.sido_embed(sido)
        gungu_embedded = self.gungu_embed(gungu)
        travel_mission_priority_embedded = self.travel_mission_priority_embed(travel_mission_priority)
        age_grp_embedded = self.age_grp_embed(age_grp)
        income_embedded = self.income_embed(income)
        #style1_embedded = self.style1_embed(style1)
        style2_embedded = self.style2_embed(style2)
        style3_embedded = self.style3_embed(style3)
        style4_embedded = self.style4_embed(style4)
        style5_embedded = self.style5_embed(style5)
        style6_embedded = self.style6_embed(style6)
        style7_embedded = self.style7_embed(style7)
        style8_embedded = self.style8_embed(style8)
        motive_embedded = self.motive_embed(motive)

        # 연속형 특성
        continuous_features = torch.cat([gender.unsqueeze(1), residence_time_min.unsqueeze(1), revisit_yn.unsqueeze(1), travel_num.unsqueeze(1), companion_num.unsqueeze(1)], dim=1)

        # 모든 임베딩 및 연속형 특성을 결합
        x = torch.cat([
            user_embedded, item_embedded, sido_embedded, gungu_embedded, travel_mission_priority_embedded,
            age_grp_embedded, income_embedded,
            style2_embedded, style3_embedded, style4_embedded,
            style5_embedded, style6_embedded, style7_embedded, style8_embedded,
            motive_embedded, continuous_features
        ], dim=-1)

        # MLP를 통해 예측값을 계산
        x = self.fc_layers(x)
        return x



In [13]:
def train(train_dataloader, optimizer, model, criterion):
    
    # 학습 루프
    for epoch in range(100):
        for batch in train_dataloader:
            features, targets = batch
            optimizer.zero_grad()

            outputs = model(features).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

def test(test_dataloader, model):
    # 모델을 평가 모드로 설정
    model.eval()

    # 예측값과 실제값을 저장할 리스트 초기화
    preds = []
    targets = []

    # 데이터로더에서 배치를 순회하며 평가
    with torch.no_grad():
        for batch in test_dataloader:
            features, target = batch
            # 모델 예측
            output = model(features).squeeze()  # 모델 출력 조정 필요 시 여기를 수정
            preds.extend(output.cpu().numpy())  # 예측값 저장
            targets.extend(target.cpu().numpy())  # 실제값 저장

    test_df['PREDICT'] = preds
    # MSE 계산
    mse = mean_squared_error(targets, preds)
    print(f'Test MSE: {mse}')
    
    return mse


In [ ]:
# 결과를 저장할 DataFrame 초기화
random_search_results_df = pd.DataFrame(columns=['w1', 'w2', 'w3', 'mse'])

best_mse = float('inf')
best_weights = None

# Random Search 설정
iterations = 100  # 탐색을 수행할 반복 횟수

for _ in range(iterations):
    # 가중치를 무작위로 선택 (합이 1이 되도록 조정)
    w1 = random.random()
    w2 = random.random() * (1 - w1)
    w3 = 1 - w1 - w2
    
    # 새로운 SCORE 계산
    df['SCORE'] = w1 * df['DGSTFN'] + w2 * df['REVISIT_INTENTION'] + w3 * df['RCMDTN_INTENTION']
    #df_test['SCORE'] = w1 * df_test['DGSTFN'] + w2 * df_test['REVISIT_INTENTION'] + w3 * df_test['RCMDTN_INTENTION']
    
    # Train-test 분할 (실제 df 사용)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Train, Test 데이터셋으로 TravelDataset 인스턴스 생성
    train_dataset = TravelDataset(train_df)
    test_dataset = TravelDataset(test_df)

    # DataLoader 설정
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_users = df['TRAVELER_ID'].nunique()
    num_items = df['VISIT_AREA_NM'].nunique()
    num_sido = df['SIDO'].nunique()
    num_gungu = df['GUNGU'].nunique()
    num_travel_mission_priority = df['TRAVEL_MISSION_PRIORITY'].nunique()
    num_age_grp = df['AGE_GRP'].nunique()
    num_income = df['INCOME'].nunique()
    num_style1 = df['TRAVEL_STYL_1'].nunique()
    num_style2 = df['TRAVEL_STYL_2'].nunique()
    num_style3 = df['TRAVEL_STYL_3'].nunique()
    num_style4 = df['TRAVEL_STYL_4'].nunique()
    num_style5 = df['TRAVEL_STYL_5'].nunique()
    num_style6 = df['TRAVEL_STYL_6'].nunique()
    num_style7 = df['TRAVEL_STYL_7'].nunique()
    num_style8 = df['TRAVEL_STYL_8'].nunique()
    num_motive = df['TRAVEL_MOTIVE_1'].nunique()
    embed_size = 16  # 임베딩 크기 설정
    
    # 모델 인스턴스 생성
    model = NCF(
        num_users=num_users, num_items=num_items, 
        num_sido=num_sido, num_gungu=num_gungu, 
        num_travel_mission_priority=num_travel_mission_priority, 
        num_age_grp=num_age_grp, num_income=num_income,
        num_style1=num_style1, num_style2=num_style2, num_style3=num_style3, 
        num_style4=num_style4, num_style5=num_style5, num_style6=num_style6, 
        num_style7=num_style7, num_style8=num_style8, num_motive=num_motive, 
        embed_size=embed_size
    )

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train(train_dataloader, optimizer, model, criterion)
    mse = test(test_dataloader, model)
    
    random_search_results_df = random_search_results_df.append({'w1': w1, 'w2': w2, 'w3': w3, 'mse': mse}, ignore_index=True)
    
    if mse < best_mse:
        best_mse = mse
        best_weights = (w1, w2, w3)
    

# DataFrame을 CSV 파일로 저장
random_search_results_df.to_csv('random_search_cf.csv', index=False)

# 최적의 MSE와 가중치 출력
print(f"Best MSE: {best_mse}")
print(f"Best weights: w1={best_weights[0]}, w2={best_weights[1]}, w3={best_weights[2]}")
    

/tmp/ipykernel_34557/2629181029.py:39: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  torch.tensor(self.residence_time_min[idx], dtype=torch.long),


Epoch 1, Loss: 0.7056321501731873
Epoch 2, Loss: 0.4360787868499756
Epoch 3, Loss: 0.6231222152709961
Epoch 4, Loss: 0.40896308422088623
Epoch 5, Loss: 0.7478969097137451
Epoch 6, Loss: 0.36712467670440674
Epoch 7, Loss: 0.41165676712989807
Epoch 8, Loss: 0.20606181025505066
Epoch 9, Loss: 0.28790345788002014
Epoch 10, Loss: 0.22555848956108093
Epoch 11, Loss: 1.021498203277588
Epoch 12, Loss: 0.5736213326454163
Epoch 13, Loss: 0.2431604266166687
Epoch 14, Loss: 0.22717301547527313
Epoch 15, Loss: 0.34455543756484985
Epoch 16, Loss: 0.06578680872917175
Epoch 17, Loss: 0.07224244624376297
Epoch 18, Loss: 0.16720430552959442
Epoch 19, Loss: 0.13267236948013306
Epoch 20, Loss: 0.21964022517204285
Epoch 21, Loss: 0.25768014788627625
Epoch 22, Loss: 0.13209745287895203
Epoch 23, Loss: 0.21425828337669373
Epoch 24, Loss: 0.09112724661827087
Epoch 25, Loss: 0.38650691509246826
Epoch 26, Loss: 0.10389889031648636
Epoch 27, Loss: 0.15350408852100372
Epoch 28, Loss: 0.07656414806842804
Epoch 29,

/tmp/ipykernel_34557/3279920816.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PREDICT'] = preds
/tmp/ipykernel_34557/1230091744.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  random_search_results_df = random_search_results_df.append({'w1': w1, 'w2': w2, 'w3': w3, 'mse': mse}, ignore_index=True)
/tmp/ipykernel_34557/2629181029.py:39: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  torch.tensor(self.residence_time_min[idx], dtype=torch.long),


Test MSE: 1.4129966497421265
Epoch 1, Loss: 0.4808848202228546
Epoch 2, Loss: 0.4547133445739746
Epoch 3, Loss: 0.46294963359832764
Epoch 4, Loss: 1.0071873664855957
Epoch 5, Loss: 0.456292986869812
Epoch 6, Loss: 0.5417840480804443
Epoch 7, Loss: 0.561427652835846
Epoch 8, Loss: 0.49334171414375305
Epoch 9, Loss: 0.32193490862846375
Epoch 10, Loss: 0.3872184157371521
Epoch 11, Loss: 0.5322033762931824
Epoch 12, Loss: 0.28539928793907166
Epoch 13, Loss: 0.3315727114677429
Epoch 14, Loss: 0.42454206943511963
Epoch 15, Loss: 0.4816496670246124
Epoch 16, Loss: 0.3004109263420105
Epoch 17, Loss: 0.18539108335971832
Epoch 18, Loss: 0.2008640319108963
Epoch 19, Loss: 0.32426509261131287
Epoch 20, Loss: 0.32064417004585266
Epoch 21, Loss: 0.15206079185009003
Epoch 22, Loss: 0.15363426506519318
Epoch 23, Loss: 0.14380109310150146
Epoch 24, Loss: 0.15929126739501953
Epoch 25, Loss: 0.06840632855892181
Epoch 26, Loss: 0.06830112636089325
Epoch 27, Loss: 0.42941442131996155
Epoch 28, Loss: 0.0458

/tmp/ipykernel_34557/3279920816.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PREDICT'] = preds
/tmp/ipykernel_34557/1230091744.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  random_search_results_df = random_search_results_df.append({'w1': w1, 'w2': w2, 'w3': w3, 'mse': mse}, ignore_index=True)
/tmp/ipykernel_34557/2629181029.py:39: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  torch.tensor(self.residence_time_min[idx], dtype=torch.long),


...

Test MSE: 1.3963947296142578
Epoch 1, Loss: 1.469306230545044
Epoch 2, Loss: 0.49889421463012695
Epoch 3, Loss: 0.48941659927368164
Epoch 4, Loss: 0.2694894075393677
Epoch 5, Loss: 0.5168073177337646
Epoch 6, Loss: 0.582704484462738
Epoch 7, Loss: 0.5716012120246887
Epoch 8, Loss: 0.3939196467399597
Epoch 9, Loss: 0.33442220091819763
Epoch 10, Loss: 0.7385677695274353
Epoch 11, Loss: 0.1917097419500351
Epoch 12, Loss: 0.33988773822784424
Epoch 13, Loss: 0.4373542368412018
Epoch 14, Loss: 0.2432234287261963
Epoch 15, Loss: 0.28664425015449524
Epoch 16, Loss: 0.3379438817501068
Epoch 17, Loss: 0.227464497089386
Epoch 18, Loss: 0.19290253520011902
Epoch 19, Loss: 0.15186600387096405
Epoch 20, Loss: 0.1136399656534195
Epoch 21, Loss: 0.16906313598155975
Epoch 22, Loss: 0.12130134552717209
Epoch 23, Loss: 0.17552119493484497
Epoch 24, Loss: 0.04778626188635826
Epoch 25, Loss: 0.17260640859603882
Epoch 26, Loss: 0.181588813662529
Epoch 27, Loss: 0.04933709651231766
Epoch 28, Loss: 0.12176202

/tmp/ipykernel_34557/3279920816.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PREDICT'] = preds
/tmp/ipykernel_34557/1230091744.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  random_search_results_df = random_search_results_df.append({'w1': w1, 'w2': w2, 'w3': w3, 'mse': mse}, ignore_index=True)
/tmp/ipykernel_34557/2629181029.py:39: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  torch.tensor(self.residence_time_min[idx], dtype=torch.long),


Test MSE: 1.3045504093170166
Epoch 1, Loss: 1.2362384796142578
Epoch 2, Loss: 0.7244035601615906
Epoch 3, Loss: 0.46132707595825195
Epoch 4, Loss: 0.540290355682373
Epoch 5, Loss: 0.9513465166091919
Epoch 6, Loss: 0.47494077682495117
Epoch 7, Loss: 0.3685254156589508
Epoch 8, Loss: 0.41566038131713867
Epoch 9, Loss: 0.5279117822647095
Epoch 10, Loss: 0.4474746286869049
Epoch 11, Loss: 0.290412575006485
Epoch 12, Loss: 0.3311861455440521
Epoch 13, Loss: 0.25327619910240173
Epoch 14, Loss: 0.39427462220191956
Epoch 15, Loss: 0.2681940793991089
Epoch 16, Loss: 0.28249889612197876
Epoch 17, Loss: 0.35747772455215454
Epoch 18, Loss: 0.21289615333080292
Epoch 19, Loss: 0.15357369184494019
Epoch 20, Loss: 0.14895141124725342
Epoch 21, Loss: 0.1878017783164978
Epoch 22, Loss: 0.309478223323822
Epoch 23, Loss: 0.11718077212572098
Epoch 24, Loss: 0.20706002414226532
Epoch 25, Loss: 0.07358602434396744
Epoch 26, Loss: 0.10857158899307251
Epoch 27, Loss: 0.10584259778261185
Epoch 28, Loss: 0.06858

/tmp/ipykernel_34557/3279920816.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PREDICT'] = preds
/tmp/ipykernel_34557/1230091744.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  random_search_results_df = random_search_results_df.append({'w1': w1, 'w2': w2, 'w3': w3, 'mse': mse}, ignore_index=True)
/tmp/ipykernel_34557/2629181029.py:39: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  torch.tensor(self.residence_time_min[idx], dtype=torch.long),


Test MSE: 1.5071858167648315
Epoch 1, Loss: 0.6851193904876709
Epoch 2, Loss: 0.3280254602432251
Epoch 3, Loss: 0.5493683218955994
Epoch 4, Loss: 0.628749430179596
Epoch 5, Loss: 0.41169437766075134
Epoch 6, Loss: 0.3979223966598511
Epoch 7, Loss: 0.38217893242836
Epoch 8, Loss: 0.23852556943893433
Epoch 9, Loss: 0.15513044595718384
Epoch 10, Loss: 0.18172889947891235
Epoch 11, Loss: 0.2754479646682739
Epoch 12, Loss: 0.133155956864357
Epoch 13, Loss: 0.13039261102676392
Epoch 14, Loss: 0.3753012418746948
Epoch 15, Loss: 0.2052554041147232
Epoch 16, Loss: 0.24757780134677887
Epoch 17, Loss: 0.08192163705825806
Epoch 18, Loss: 0.19813016057014465
Epoch 19, Loss: 0.20780760049819946
Epoch 20, Loss: 0.26126420497894287
Epoch 21, Loss: 0.3231087327003479
Epoch 22, Loss: 0.1102854385972023
Epoch 23, Loss: 0.20712517201900482
Epoch 24, Loss: 0.20574702322483063
Epoch 25, Loss: 0.27417615056037903
Epoch 26, Loss: 0.15102458000183105
Epoch 27, Loss: 0.0869087278842926
Epoch 28, Loss: 0.0445352

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 모델에서 사용자 임베딩 레이어 추출
user_embeddings = model.user_embed.weight.data.numpy()

# 모든 사용자 간의 코사인 유사도 계산
similarity_matrix = cosine_similarity(user_embeddings)

# 특정 사용자 ID에 대한 유사도 점수 검색 (예: 0번 사용자)
# 사용자 ID와 그에 해당하는 숫자 인덱스 매핑 생성
user_id_to_index = {user_id: index for index, user_id in enumerate(df['TRAVELER_ID'].astype('category').cat.categories)}
index_to_user_id = {index: user_id for index, user_id in enumerate(df['TRAVELER_ID'].astype('category').cat.categories)}

# 문자열 사용자 ID를 숫자 인덱스로 변환
id = 'a015688'
user_index = user_id_to_index[id]
user_similarity_scores = similarity_matrix[user_index]

# 자기 자신을 제외한 가장 유사한 사용자 상위 N명 찾기
top_n_indices = np.argsort(user_similarity_scores)[::-1][1:6]  # 자기 자신을 제외하고 상위 5명

# 숫자 인덱스를 사용자 ID로 변환
top_n_user_ids = [index_to_user_id[i] for i in top_n_indices]

print(f"사용자 a015688와 가장 유사한 사용자 ID:", top_n_user_ids)
print("유사도 점수:", user_similarity_scores[top_n_indices])